In [ ]:
import arviz as az
import matplotlib
import numpy as np
import pymc as pm

## A Bayesian coin flip example

In [ ]:
obs = np.array([True, True, True, True, False])
obs

In [ ]:
with pm.Model() as m1:
    p = pm.Beta('p', alpha=1, beta=1)
    likelihood = pm.Bernoulli('likelihood', p=p, observed=obs)

In [ ]:
pm.model_to_graphviz(m1)

In this diagram, grey refers to observed variables. White refers to hidden ('latent') variables. The rectangle represents duplication of observations. So we have 5 observed Bernoulli trials. These are parametrized by a probability that itself is drawn from a distribution. We don't know what the p is, but we have an idea of the distribution.

Another way of looking at this is our model tries to reconcile two pieces of information. Firstly we have a prior which reflects knowledge we have before observing the trials. Secondly we have some observations. The model describes the best way of reconciling these two pieces of information.

In [ ]:
with m1:
    trace = pm.sample(draws=1000, tune=1000)

In [ ]:
az.plot_trace(trace, var_names=['p'], combined=True)